<h1>Heroes Of Pymoli Data Analysis</h1>
<ul>
    <li>Most Popular:  <b>Betrayal, Whisper of Grieving Widows</b></li>
    <li>Age Group Spending the Most:  <b>20-24</b></li>
    <li>Gender that Purchase the Most:  <b>Male</b></li>
</ul>

In [20]:
import pandas as pd
import numpy as np

In [30]:
# Load the json file to a dataframe.
# This dataframe will be used as the database.
f = "purchase_data.json"
df = pd.read_json(f, orient='records')
df.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [32]:
# Players with multiple purchases.

# Get players (including duplicates).
series_SN = df.loc[:,'SN'] 

# Create a dictionary of unique players with there number of purchases
dict_SN = dict(series_SN.groupby(series_SN).count())

# Create a dictionary containing the column name for the count.
columns_SN = {'Count'}

# Create a dataframe of unique players and their number of purchases.
df_SN = pd.DataFrame(dict_SN, columns_SN)

# Swap axis for presentation and filtering
df_SN = df_SN.T

# Select those that have more tha one purchase.
df_SN = df_SN.loc[df_SN.Count > 1]

# Select the index SN's of the players with multiple purchases.
index_SN = df_SN.index

# Put the index into a list to used as a filer.
list_SN = list(index_SN.values)

# Display the puchases for players that have more than one purchase.
df.loc[df.SN.isin(list_SN)].sort_values(by=['SN', 'Item ID']).head(10)

,Age,Gender,Item ID,Item Name,Price,SN
308,37,Male,79,"Alpha, Oath of Zeal",2.88,Aduephos78
431,37,Male,92,Final Critic,1.36,Aduephos78
377,37,Male,174,Primitive Blade,2.46,Aduephos78
721,26,Male,39,"Betrayal, Whisper of Grieving Widows",2.35,Aeduera68
224,26,Male,106,Crying Steel Sickle,2.29,Aeduera68
647,26,Male,156,Soul-Forged Steel Shortsword,1.16,Aeduera68
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
529,38,Male,172,Blade of the Grave,1.69,Aelalis34
637,20,Male,18,"Torchlight, Bond of Storms",1.77,Aeliriam77
359,20,Male,32,Orenmir,4.95,Aeliriam77


<h2>Player Count</h2>

In [33]:
# Player Count

# Get a series of unique players and then count the number of players.
unique_player_count = df.loc[:,'SN'].unique().size

# For grins and giggles, count the total number of players in the data.
all_player_count = df.loc[:,'SN'].size

# Build a dictionary of players and their count to be used in a dataframe.
data_player_count = {"Unique":unique_player_count, "All":all_player_count}

# Build a list of headers to be used in the dataframe.
columns_player_count = ['Total Players']

# Create the dataframe of players and their count.
df_player_count = pd.DataFrame(data_player_count, columns_player_count)

# Flip the axis for display.
df_player_count = df_player_count.T

# Display the players and counts for unique players and all players.
df_player_count.head()

,Total Players
All,780
Unique,573


<h2>Purchasing Analysis (Total)</h2>

In [34]:
# Purchasing Analysis (Total)

# Callculate total revenue.
total_revenue = df['Price'].sum()

# Calculate number of purchases.
number_of_purchases = df['Price'].count()

# Calculate the average price of a purchase.
average_price = total_revenue / number_of_purchases

# Calculate the number of unique items.
number_of_unique_items = df['Item ID'].unique().size

# Create a dataframe with just the number of unique items.
total_dict = {}
total_dict["Number of Unique Items"] = number_of_unique_items
total_cols = [" "]
df_total = pd.DataFrame(total_dict, total_cols)

# Add average price, number of purchases, and total revenue to the dataframe.
df_total['Average Price'] = average_price
df_total["Number of Purchases"] = number_of_purchases
df_total["Total Revenue"] = total_revenue

# Format the currency fields of average price and total revenue,.
df_total['Average Price'] = df_total['Average Price'].map('${:,.2f}'.format)
df_total["Total Revenue"] = df_total["Total Revenue"].map('${:,.2f}'.format)

# Display the dataframe with the calculated and formated amounts.
df_total

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
,183,$2.93,780,"$2,286.33"


<h2>Gender Demographics</h2>

In [35]:
# Gender Demographics

# Get all unique players and their gender.
df_gender = df[['Gender', 'SN']].drop_duplicates()

# Calculate the number of players for the percentage calculation.
total_population = df_gender['SN'].count()

# Group the players by gender and count the number in each group.
df_gender = df_gender.groupby('Gender').count()

# Rename the column with count of players to total players.
df_gender = df_gender.rename(columns={"SN": "Total Count"})

# Calculate the percentage of players in each group.
df_gender['Percentage of Players'] = round((df_gender['Total Count'] / total_population) * 100, 2)

# Rearrange the columns to meet display requirements.,
df_gender = df_gender.reindex(columns=['Percentage of Players', 'Total Count'])

# Rearrange the rows do display the number of players is descending order.
df_gender = df_gender.sort_values(by='Total Count', ascending=False)

# Display the player count by gender.
df_gender

,Percentage of Players,Total Count
Gender,,
Male,81.15,465
Female,17.45,100
Other / Non-Disclosed,1.40,8


<h2>Purchasing Analysis (Gender)</h2>

In [36]:
# Purchasing Analysis (Gender)

# Get the gender and price for each purchase.
df_purgen = df[['Gender', 'Price']]

# Let the computer do the work:  Aggregate by gender.
df_purgen = df_purgen.groupby('Gender').agg({'Price': ['count', 'mean', 'sum', 'median']})

# Aggregation creates a multiple-level index with price on top.  Delete it.,
df_purgen.columns = df_purgen.columns.droplevel(0)

# Sort results by total purchase value.
df_purgen = df_purgen.sort_values(by="sum", ascending=False)

# Create a dictionary of new column headings to meet requirements.
column_names = {}
column_names['count'] = 'Purchase Count'
column_names['mean'] = 'Average Purchase Price'
column_names['sum'] = "Total Purchase Value"
column_names['median'] = "Normalized Totals"

# Rename the column headings in the dataframe.
df_purgen = df_purgen.rename(columns=column_names)

# Format currency fields
df_purgen['Average Purchase Price'] = df_purgen['Average Purchase Price'].map('${:,.2f}'.format)
df_purgen['Total Purchase Value'] = df_purgen['Total Purchase Value'].map('${:,.2f}'.format)
df_purgen['Normalized Totals'] = df_purgen['Normalized Totals'].map('${:,.2f}'.format)

# Display the dataframe with the analysis
df_purgen

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Male,633,$2.95,"$1,867.68",$2.91
Female,136,$2.82,$382.91,$2.62
Other / Non-Disclosed,11,$3.25,$35.74,$3.73


<h2>Age Demographics</h2>

In [37]:
# Age Demographics

# Create a datafram of users and their ages.
df_age = pd.DataFrame(df[['Age','SN']].to_dict())

# Remove the duplicates, leaving unique users and their ages
df_age = df_age.drop_duplicates()

# Count the the number of players for calculating percentages.
total_players = df_age['SN'].count()

# Put the players into bins by age.
bins = [0, 10, 14, 19, 24, 29, 34, 39, 100]
group_names = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']
df_age['Group'] = pd.cut(df_age['Age'], bins, labels=group_names)

# Aggregate the number of users in each age group.
df_age = df_age.groupby('Group').agg({'SN': ['count']})

# The aggregation created a two-level column index.  Remove the top level.
df_age.columns = df_age.columns.droplevel(0)

# Rename the count colun to total count.
df_age = df_age.rename(columns={'count':'Total Count'})

# Calculate the percentage of players in each group
df_age['Percentage of Players'] = round((df_age['Total Count'] / total_players) * 100, 2)

# Arrange the columns per requirements
df_age = df_age.reindex(columns=['Percentage of Players', 'Total Count'])

# Display the analysis.
df_age

,Percentage of Players,Total Count
Group,,
<10,3.84,22
10-14,3.49,20
15-19,17.45,100
20-24,45.20,259
25-29,15.18,87
30-34,8.20,47
35-39,4.71,27
40+,1.92,11


<h2>Purchasing Analysis (Age)</h2>

In [38]:
# Purchasing Analysis (Age)

# Create a new dataframe age and price for sales.
df_agepur = pd.DataFrame(df[['Age','Price']].to_dict())

# Put the sales into bins by age.
bins = [0, 10, 14, 19, 24, 29, 34, 39, 100]
group_names = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']
df_agepur['Group'] = pd.cut(df_agepur['Age'], bins, labels=group_names)

# Aggregate sales by age group.
df_agepur = df_agepur.groupby('Group').agg({'Price': ['count', 'mean', 'sum', 'median']})

# The aggregation created a two-level index.  Remove the top level
df_agepur.columns = df_agepur.columns.droplevel(0)

# Create a dictionary of new column headings.
column_names = {}
column_names['count'] = 'Purchase Count'
column_names['mean'] = 'Average Purchase Price'
column_names['sum'] = "Total Purchase Value"
column_names['median'] = "Normalized Totals"

# Apply the new column names to the dataframe.
df_agepur = df_agepur.rename(columns=column_names)

# Format the currency fields.
df_agepur['Average Purchase Price'] = df_agepur['Average Purchase Price'].map('${:,.2f}'.format)
df_agepur['Total Purchase Value'] = df_agepur['Total Purchase Value'].map('${:,.2f}'.format)
df_agepur['Normalized Totals'] = df_agepur['Normalized Totals'].map('${:,.2f}'.format)

# Display the analysis.
df_agepur

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Group,,,,
<10,32,$3.02,$96.62,$3.34
10-14,31,$2.70,$83.79,$2.37
15-19,133,$2.91,$386.42,$2.72
20-24,336,$2.91,$978.77,$2.87
25-29,125,$2.96,$370.33,$3.14
30-34,64,$3.08,$197.25,$3.06
35-39,42,$2.84,$119.40,$2.69
40+,17,$3.16,$53.75,$2.91


<h2>Top Spenders</h2>

In [40]:
# Top Spenders

# Create a data frame of users and the price they paid for a purchase
df_spenders = pd.DataFrame(df[['SN','Price']].to_dict())

# Aggregate purchase activity by player.
df_spenders = df_spenders.groupby('SN').agg({'Price':['count', 'mean', 'sum']})

# The aggregation created a two-level index.  Remove the top level.
df_spenders.columns = df_spenders.columns.droplevel(0)

# Sort the sales in descending order so the players spending the most bubble to the top.
df_spenders = df_spenders.sort_values(by='sum', ascending=False)

# Create a dictionary of new column names.
column_names = {}
column_names['count'] = 'Purchase Count'
column_names['mean'] = 'Average Purchase Price'
column_names['sum'] = "Total Purchase Value"

# Apply the new column names to the data frame.
df_spenders = df_spenders.rename(columns=column_names)

# Format the currency fields.
df_spenders['Average Purchase Price'] = df_spenders['Average Purchase Price'].map('${:,.2f}'.format)
df_spenders['Total Purchase Value'] = df_spenders['Total Purchase Value'].map('${:,.2f}'.format)

# Rename the row label from SN to Player.
df_spenders.index.names = ["Player"]

# Display the analysis.
df_spenders.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
Player,,,
Undirrala66,5,$3.41,$17.06
Saedue76,4,$3.39,$13.56
Mindimnya67,4,$3.18,$12.74
Haellysu29,3,$4.24,$12.73
Eoda93,3,$3.86,$11.58


<h2>Most Popular Items</h2>

In [41]:
# Most Popular Items

# Create a dataframe of items and their prices.
df_popular = pd.DataFrame(df[['Item ID', 'Item Name', 'Price']].to_dict())

# Aggregate the items for count, average price, and total revenue.
df_popular = df_popular.groupby(['Item ID', 'Item Name']).agg({'Price':['count', 'mean', 'sum']})

# The aggregation results in a two-level column index.  Drop the top level.
df_popular.columns = df_popular.columns.droplevel(0)

# Sort the results by descending count so the most popular items will bubble to the top.
df_popular = df_popular.sort_values(by=['count', 'sum'], ascending=False)

# Create a dictionary of new column names.
column_names = {}
column_names['count'] = 'Purchase Count'
column_names['mean'] = 'Item Price'
column_names['sum'] = "Total Purchase Value"

# Apply the new column names to the dataframe.
df_popular = df_popular.rename(columns=column_names)

# Create a list of columns to be used to rearrange columns.
column_sequence = []
column_sequence.append('Purchase Count')
column_sequence.append('Item Price')
column_sequence.append('Total Purchase Value')

# Rearrange columns per requirements.
df_popular = df_popular.reindex(columns=column_sequence)

# Format the currency values.
df_popular['Item Price'] = df_popular['Item Price'].map('${:,.2f}'.format)
df_popular['Total Purchase Value'] = df_popular['Total Purchase Value'].map('${:,.2f}'.format)

# Display the analysis
df_popular.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
84,Arcane Gem,11,$2.23,$24.53
34,Retribution Axe,9,$4.14,$37.26
31,Trickster,9,$2.07,$18.63
13,Serenity,9,$1.49,$13.41


In [ ]:
<h2>Most Profitable Items</h2>

In [42]:
# Most Profitable

# The most profitable is the same data as most popular.
# The difference is that the data is sorted so the most profitable would bubble to the top.
df_popular.sort_values(by=['Total Purchase Value', 'Purchase Count'])

# Display the analysis.
df_popular.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
84,Arcane Gem,11,$2.23,$24.53
34,Retribution Axe,9,$4.14,$37.26
31,Trickster,9,$2.07,$18.63
13,Serenity,9,$1.49,$13.41
